In [126]:
import scipy 
import sklearn
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial

In [127]:
#Extracting the data from data and labels files using pandas
data = pd.read_csv('data.csv')
labels = pd.read_csv('label.csv',names=['label'],header=None)

In [128]:
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [129]:
labels.head()

,label
0,7
1,2
2,1
3,0
4,4


In [130]:
#splitting the data for training and testing
from sklearn.model_selection import train_test_split
train_d, test_d = train_test_split( data, test_size=0.08, random_state=42)
train_l, test_l = train_test_split( labels, test_size=0.08, random_state=42)

In [131]:
class KMeans:

    def SSE_score(self, centroid_value, centroid_dict,data):
        sse_count = 0
        for i in centroid_dict:
            clust_sse = 0
            # np.sum()
            for j in centroid_dict[i]:
                datp = list(data.iloc[int(j)])
                for x,y in zip(centroid_value[i],datp):
                    clust_sse += (x-y)**2
            sse_count+=clust_sse
        return sse_count    
    
    def Centroid_intz(self,data,k):
        z = data.shape[0]
        centroid_value={}
        for i in range(k):
            rd = np.random.randint(0, z-1)
            centroid_value[i] = data.iloc[rd]
        return centroid_value
    
    def jaccard_sim(self,centroid, datp):
        intersection = len(list(set(centroid).intersection(datp)))
        union = (len(set(centroid)) + len(set(datp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #mode 1 is eucledian
        #mode 2 is jaccard
        #mode 3 is cosine
        centroid_value = self.Centroid_intz(data,K)
        new_centroid_value = {}
        cnt = 0
        centroid_dict = {}
        convergence = False
        while((cnt<max_iter) and not convergence):
            
            for i in list(centroid_value.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value[j])  for j in centroid_value]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_sim(list(x),centroid_value[j]) for j in centroid_value]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value[j]))  for j in centroid_value]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            tot_cur=-1
            for i in centroid_value:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                tot_cur= max(change, tot_cur)
                
            print("Total number of Iterations count ",cnt,": ",tot_cur)
            
            cnt+=1
            if (tot_cur<10):
                convergence = True
                break
        
        return centroid_value,centroid_dict
    

In [132]:
def clust_predic(cs, S, labels):
    
    cluster_labels = np.zeros(10,dtype=int)
    for k in cs:
        labels_of_points = []
        for point in S[k]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[k] = max(counter, key=counter.get)
        except:
            cluster_labels[k] = np.random.randint(0,9)
    return cluster_labels

In [133]:
def jaccard_sim(centroid, datp):
        intsec = len(list(set(centroid).intsec(datp)))
        unionValue = (len(set(centroid)) + len(set(datp))) - intsec
        return float(intsec) / unionValue

In [134]:
def accuracy(centroids, centroid_Labels, dat_test,lab_tru, mode=1):
    y_true = list(lab_tru['label']);
    y_pred = []
    for index in range(dat_test.shape[0]):
        featureset = dat_test.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_sim(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = dat_test.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [135]:
model_one = KMeans()
centroids_one,clusters_one = model_one.train_Kmeans(data,10, max_iter=100,mode=1)

Total number of Iterations count  0 :  26670.071022727272
Total number of Iterations count  1 :  4118.685893459664
Total number of Iterations count  2 :  3426.175467775467
Total number of Iterations count  3 :  2462.513187426242
Total number of Iterations count  4 :  2047.5241006230317
Total number of Iterations count  5 :  1836.725665042743
Total number of Iterations count  6 :  1443.9091456418482
Total number of Iterations count  7 :  1308.6227757515474
Total number of Iterations count  8 :  1189.261174242424
Total number of Iterations count  9 :  824.2360361491149
Total number of Iterations count  10 :  874.2109484064513
Total number of Iterations count  11 :  672.9876558026299
Total number of Iterations count  12 :  751.0912992701806
Total number of Iterations count  13 :  600.9899389192648
Total number of Iterations count  14 :  474.5313130827645
Total number of Iterations count  15 :  425.57299078195655
Total number of Iterations count  16 :  369.0048134777379
Total number of Ite

In [136]:
Sse_euclid = model_one.SSE_score(centroids_one,clusters_one,data)

In [137]:
print("SSE of euclidean:",Sse_euclid)

SSE of euclidean: 25429090251.489662


In [138]:
cluster_labelOne = clust_predic(centroids_one,clusters_one,labels)
cluster_labelOne

array([3, 3, 7, 0, 9, 2, 9, 8, 1, 5])

In [139]:
Accuracy_Euclidean = accuracy(centroids_one, cluster_labelOne,test_data,test_labels)
Accuracy_Euclidean

0.085

In [140]:
model_two = KMeans()
centroids_two,cluster_two = model2.train_Kmeans(data,10, max_iter=100,mode=2)
Sse_jacc = model2.SSE_score(centroids_two,cluster_two,data)

Total number of Iterations count  0 :  37728.310725552044
Total number of Iterations count  1 :  5641.902471302115
Total number of Iterations count  2 :  2102.5249893566834
Total number of Iterations count  3 :  1675.5653493920818
Total number of Iterations count  4 :  1748.5207437870858
Total number of Iterations count  5 :  2105.5256722833165
Total number of Iterations count  6 :  948.2888425309552
Total number of Iterations count  7 :  782.6303025970467
Total number of Iterations count  8 :  831.6921120683496
Total number of Iterations count  9 :  847.3218437294374
Total number of Iterations count  10 :  724.1598015081822
Total number of Iterations count  11 :  671.4036226684299
Total number of Iterations count  12 :  328.771642583229
Total number of Iterations count  13 :  0.0


In [141]:
print("SSE for jaccard",Sse_jacc)

SSE for jaccard 34361687572.938736


In [142]:
cluster_label_two = clust_predic(centroids_two,cluster_two,labels)
cluster_label_two

array([1, 6, 5, 5, 3, 2, 2, 1, 1, 0])

In [143]:
Accuracy_Jaccard = accuracy(centroids_two, cluster_label_two,test_data,test_labels)
Accuracy_Jaccard

0.1075

In [144]:
model_three = KMeans()
centroids_three,clusters_three = model_three.train_Kmeans(data,10, max_iter = 100,mode=3)

Total number of Iterations count  0 :  31159.651564185544
Total number of Iterations count  1 :  5506.3951061532925
Total number of Iterations count  2 :  3593.805377509124
Total number of Iterations count  3 :  4963.302078878507
Total number of Iterations count  4 :  3069.892127179063
Total number of Iterations count  5 :  1171.1084997613766
Total number of Iterations count  6 :  1124.891007653061
Total number of Iterations count  7 :  981.0253511290446
Total number of Iterations count  8 :  871.423940621154
Total number of Iterations count  9 :  559.6601744601744
Total number of Iterations count  10 :  537.9095915678522
Total number of Iterations count  11 :  416.34609205421395
Total number of Iterations count  12 :  404.9430598637042
Total number of Iterations count  13 :  331.80318334863784
Total number of Iterations count  14 :  396.29743348883153
Total number of Iterations count  15 :  367.15595235175886
Total number of Iterations count  16 :  363.5693551256235
Total number of It

In [145]:
Sse_cos = model_three.SSE_score(centroids_three,clusters_three,data)

In [146]:
print("SSE of euclidean :",Sse_euclid)
print("SSE of Jacard :",Sse_jacc)
print("SSE of Cosine :",Sse_cos)

SSE of euclidean : 25429090251.489662
SSE of Jacard : 34361687572.938736
SSE of Cosine : 25613952004.39616


In [147]:
cluster_labels_three = clust_predic(centroids_three,clusters_three,labels)
cluster_labels_three

array([2, 7, 5, 9, 3, 3, 6, 1, 0, 8])

In [148]:
Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Cosine

0.07875

In [149]:
print("Accuracy of Euclidean:",Accuracy_Euclidean)
print("Accuracy of Jacard:",Accuracy_Jaccard)
print("Accuracy of Cosine :",Accuracy_Cosine)

Accuracy of Euclidean: 0.085
Accuracy of Jacard: 0.1075
Accuracy of Cosine : 0.07875
